In [1]:
%matplotlib inline

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
from tensorflow.keras import layers, models

import numpy as np

2024-07-27 16:55:37.414689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 16:55:37.457450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 16:55:37.469531: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_images_path = './data/train'
train_masks_path = './data/train-masks'

In [4]:
# there is no annotations for this file in .json
# TODO: make check on this file if exists
if not os.remove('./data/train/2369_jpg.rf.8b8afa9d79c61fa42ca128c940b9cbc0.jpg'):
    pass

FileNotFoundError: [Errno 2] No such file or directory: './data/train/2369_jpg.rf.8b8afa9d79c61fa42ca128c940b9cbc0.jpg'

In [5]:
from data_clean import DataClean

dc = DataClean()
dc.resize_img_dir_128(train_images_path)
dc.resize_img_dir_128(train_masks_path)

Generating masks...
Generated 1502 masks
Resized directory with name `./data/train` to 128x128 images
Resized directory with name `./data/train-masks` to 128x128 images


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images_from_folder(folder, target_size=(128, 128), grayscale=False):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.coco.json'):
            continue
        img_path = os.path.join(folder, filename)
        try:
            img = load_img(img_path, target_size=target_size, color_mode='grayscale' if grayscale else 'rgb')
            img = img_to_array(img)
            images.append(img)
        except:
            print(f"Cannot identify image file {img_path}. Skipping.")
    return np.array(images)

In [ ]:
X_train = load_images_from_folder(train_images_path)
y_train = load_images_from_folder(train_masks_path, target_size=(128, 128), grayscale=False)

In [ ]:
X_train, y_train

In [ ]:
if len(X_train) != len(y_train):
    raise ValueError(f"Number of training images ({len(X_train)}) and masks ({len(y_train)}) do not match.")

In [ ]:
X_train = X_train / 255.0
y_train = y_train / 255.0
y_train = np.round(y_train) # Перетворення масок у бінарний формат

In [ ]:
X_train.shape, y_train.shape

In [ ]:
def u_net_model(input_size=(128, 128, 1)):
    inputs = layers.Input(input_size)

    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    p5 = layers.MaxPooling2D((2, 2))(c5)

    u6 = layers.Conv2DTranspose(256, (2,2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2,2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2,2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2,2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    res = layers.Conv2D(1, (1, 1), activation='sigmoid')(c1)
    return models.Model(inputs=[inputs], outputs=[res])

In [ ]:
model = u_net_model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.1)

In [ ]:
model.evaluate(X_test, y_test)